In [9]:
import os
import json
from dotenv import load_dotenv
from openai import OpenAI
from openai.types.chat import ChatCompletionToolParam
from tavily import TavilyClient

load_dotenv("../.env")
client = OpenAI(api_key=os.environ['API_KEY'])
TAVILY_API_KEY = os.environ['TAVILY_API_KEY']
MODEL_NAME = "gpt-4o-mini"

In [10]:
# 検索結果を返す関数の作成
def get_search_result(question):
    client = TavilyClient(api_key=TAVILY_API_KEY)
    response = client.search(question)
    return json.dumps({"result": response["results"]})


In [11]:
# テスト
ret = get_search_result("奄美市でのイベントを教えて")
json.loads(ret)

{'result': [{'url': 'https://www.jalan.net/event/cit_462220000/',
   'title': '奄美市のイベントランキング - じゃらんnet',
   'content': '「あまみシマ博覧会」と題し、奄美群島の自然や食、文化に親しめる体験プログラムが各地で開催されます。大島紬の泥染めや、夜光貝磨きなどの伝統工芸、郷土料理作り、奄美',
   'score': 0.79119295,
   'raw_content': None},
  {'url': 'https://www.neriyakanaya.jp/contents/oshima_exp',
   'title': '奄美大島の観光・イベント情報 - ねりやかなや',
   'content': '紬の日（奄美市） 1月5日は「紬の日」で、昭和53年に本場奄美大島紬の振興による豊かなまちづくりを推進するために制定。 さまざまなイベントを開催します。 紬の日（奄美市）.',
   'score': 0.7237202,
   'raw_content': None},
  {'url': 'https://x.com/amamaru_event',
   'title': '奄美 まるごとイベント情報局（AMA-MARU ... - X',
   'content': '奄美大島◦群島に関連する色んなイベントをご紹介します♪ 奄美の豆知識や奄美あるあるなどもお届けしますよ！奄美好きさんフォローしてね！管理人は奄美大島出身のしっし',
   'score': 0.67885995,
   'raw_content': None},
  {'url': 'https://amakannext.com/?cat=8',
   'title': '企画・イベント - 奄美観光グループの情報サイト「奄観NEXT」',
   'content': '／本日は 花火大会です＼ ... 奄美市市制施行20周年記念「第62回奄美まつり」2日目は、花火大会です！ 花火だよ！全員集合場所：御殿浜公園時間帯：17:00～22:00主催：あまみエフエム',
   'score': 0.6786044,
   'raw_content': None},
  {'url'

In [17]:
def define_tools():
    print("------define_tools(ツール定義)------")
    return [
        ChatCompletionToolParam(
            {
                "type": "function",
                "function": {
                    "name": "get_search_result",
                    "description": "最近一ヵ月のイベント開催予定などネット検索が必要な場合に、質問文の検索結果を取得する",
                    "parameters": {
                        "type": "object",
                        "properties": {
                            "question": {"type": "string", "description": "質問文"}
                        },
                        "required": ["question"],
                    },
                },
            }
        )
    ]

In [ ]:
# 言語モデルへの質問を行う関数
def ask_question(question, tools):
    response = client.chat.completions.create(
        model=MODEL_NAME,
        messages=[{"role": "user", "content": question}],
        tools=tools,
        tool_choice="auto",
    )
    return response


In [ ]:
# ツール呼び出しが必要な場合の処理を行う関数

def handle_tool_call(response, question):
    # 関数の実行と結果取得
    tool = response.choices[0].message.tool_calls[0]
    function_name = tool.function.name
    arguments = json.loads(tool.function.arguments)
    function_response = globals()[function_name](**arguments)

    # 関数の実行結果をmessagesに加えて再度言語モデルを呼出
    response_after_tool_call = client.chat.completions.create(
        model=MODEL_NAME,
        messages=[
            {"role": "user", "content": question},
            response.choices[0].message,
            {
                "tool_call_id": tool.id,
                "role": "tool",
                "content": function_response,
            },
        ],
    )
    return response_after_tool_call


In [ ]:
# ユーザーからの質問を処理する関数
"""
def process_response(question, tools):
    response = ask_question(question, tools)

    if response.choices[0].finish_reason == "tool_calls":
        # ツール呼び出し
        final_response = handle_tool_call(response, question)
        return final_response.choices[0].message.content.strip()
    else:
        # 直接回答
        return response.choices[0].message.content.strip()
"""
def process_response(question, tools):
    response = ask_question(messages, tools)

    if response.choices[0].finish_reason == "tool_calls":
        # ツール呼び出し
        final_response = handle_tool_call(response, question)
        return final_response.choices[0].message.content.strip()
    else:
        # 直接回答
        return response.choices[0].message.content.strip()

In [18]:
tools = define_tools()

# 直接回答
question = "東京都と沖縄県はどちらが広いですか？"
response_message = process_response(question, tools)
print(response_message)

------define_tools(ツール定義)------
面積で比較すると、沖縄県の方が広いです。

- 東京都の面積は約2,194平方キロメートルです。
- 沖縄県の面積は約2,271平方キロメートルです。

したがって、沖縄県は東京都よりも広い地域を持っています。


In [26]:
tools = define_tools()

# ツール呼出が必要な質問
question = "東京駅のイベントについて、最近1ヶ月以内の検索結果を教えてください"
response_message = process_response(question, tools)
print(response_message)

------define_tools(ツール定義)------
最近1ヶ月以内に東京駅で行われるイベントに関する情報は以下の通りです：

1. **Tokyo Innovation Base (TIB) 展示**
   - 開催日: 1ヶ月間
   - 内容: 東京駅近くのTokyo Innovation Baseにて行われる展示です。詳細な内容は[こちら](https://www.instagram.com/p/C8LjI59hw7N/)を参照してください。

2. **COMIC CITY VEGA 2025**
   - 開催日: 8月31日
   - 内容: 東京豊洲で開催されるイベント。入場券の予告などが行われています。詳しくは[こちら](https://www.facebook.com/manyo.toyosu/posts/)をチェックしてください。

3. **HOKUSAI: ANOTHER STORY in TOKYO**
   - 時期: 2025年2月1日から8月11日
   - 内容: 葛飾北斎に関する展示イベント。美術の世界を体感できる展覧会です。詳細は[こちら](https://www.walkerplus.com/top/ar0313/)から。

4. **和楽器コンサート**
   - 開催日: 2025年4月18日から9月23日予定
   - 内容: 和楽器にフォーカスを当てたコンサートシリーズ。詳細は[こちら](https://www.enjoytokyo.jp/event/list/)を参照してください。

この他にもさまざまなイベントが東京駅周辺で計画されていますので、気になる方は公式サイトやSNSで最新情報をチェックすることをお勧めします。


In [25]:
# チャットボットへの組み込み
tools = define_tools()

messages=[]

while(True):
    # ユーザーからの質問を受付
    question = input("メッセージを入力:")
    # 質問が入力されなければ終了
    if question.strip()=="":
        break
    display(f"質問:{question}")

    # メッセージにユーザーからの質問を追加
    messages.append({"role": "user", "content": question.strip()})
    # やりとりが8を超えたら古いメッセージから削除
    if len(messages) > 8:
        del_message = messages.pop(0)

    # 言語モデルに質問
    response_message = process_response(question, tools)

    # メッセージに言語モデルからの回答を追加
    print(response_message, flush=True)
    messages.append({"role": "assistant", "content": response_message})

print("\n---ご利用ありがとうございました！---")

------define_tools(ツール定義)------


'質問:soujino'

It appears that "Soujino" could refer to a variety of topics, primarily associated with individuals or creative content online. Here are some notable mentions:

1. **Social Media**: [Soujino on Twitter](https://twitter.com/Soujino) is described as a Pokemon enthusiast and a creator in the realm of decentralized finance (DeFi) and digital art (JPEGs).

2. **Art Profile**: You can find [Soujino's artwork on DeviantArt](https://www.deviantart.com/soujino), where they share their creations.

3. **Anime Series**: There is a mention of "Souji no Kanojo," which seems to be an anime or related content available with English subtitles on platforms like [BiliBili](https://www.bilibili.tv/en/video/4796280060777984).

4. **Facebook**: A person named [Soujino Jikankameyama](https://m.facebook.com/soujino.jikankameyama.7/) is also present on Facebook, where you can connect with them.

5. **One Piece Fan Content**: There is a reference to a fictional item in the One Piece universe, the [Souji Souji n